In [89]:
import pandas as pd
training_data_df = pd.read_csv('keras_train.csv')
testing_data_df = pd.read_csv('keras_test.csv')

In [90]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
scaled_training = scaler.fit_transform(training_data_df)
scaled_testing = scaler.transform(testing_data_df)

print("mutiplying by {:.10f} and adding {:.6f}".format(scaler.scale_[6],scaler.min_[6]))


mutiplying by 0.0214224507 and adding 0.000000


In [91]:
scaled_training_df = pd.DataFrame(scaled_training,columns=training_data_df.columns.values)
scaled_testing_df = pd.DataFrame(scaled_testing,columns=testing_data_df.columns.values)

scaled_training_df.to_csv("Options_data_training_scaled.csv", index=False)
scaled_testing_df.to_csv("Options_data_testing_scaled.csv", index=False)

In [101]:
import pandas as pd
from keras.models import Sequential
from keras.layers import *

training_data_df = pd.read_csv("Options_data_training_scaled.csv")
X = training_data_df.drop('premium',axis=1).values
Y = training_data_df[['premium']].values

In [102]:
model = Sequential()
model.add(Dense(50,input_dim=6,activation='relu'))
model.add(Dense(100,activation='relu'))
model.add(Dense(100,activation='relu'))
model.add(Dense(50,activation='relu'))
model.add(Dense(1,activation='linear'))
model.compile(loss='mean_squared_error',optimizer='adam')

In [103]:
# Train the model
model.fit(
    X,
    Y,
    epochs=100,
    shuffle=True,
    verbose=2
)

#Load the separate test dataset
test_data_df = pd.read_csv("Options_data_testing_scaled.csv")
X_test = test_data_df.drop('premium',axis=1).values
Y_test = test_data_df[['premium']].values

test_error_rate = model.evaluate(X_test, Y_test, verbose=0)
print("The mean squared error (MSE) for the test data set is: {}".format(test_error_rate))

Epoch 1/100
7s - loss: 8.8076e-05
Epoch 2/100
7s - loss: 1.5546e-05
Epoch 3/100
7s - loss: 1.0699e-05
Epoch 4/100
7s - loss: 7.6085e-06
Epoch 5/100
7s - loss: 6.9216e-06
Epoch 6/100
7s - loss: 5.7605e-06
Epoch 7/100
7s - loss: 4.7355e-06
Epoch 8/100
7s - loss: 3.6745e-06
Epoch 9/100
7s - loss: 3.0126e-06
Epoch 10/100
7s - loss: 3.4702e-06
Epoch 11/100
7s - loss: 3.1953e-06
Epoch 12/100
7s - loss: 2.4213e-06
Epoch 13/100
7s - loss: 2.8021e-06
Epoch 14/100
7s - loss: 2.2795e-06
Epoch 15/100
7s - loss: 2.7270e-06
Epoch 16/100
7s - loss: 3.4167e-06
Epoch 17/100
7s - loss: 2.5775e-06
Epoch 18/100
7s - loss: 1.8948e-06
Epoch 19/100
7s - loss: 2.4153e-06
Epoch 20/100
7s - loss: 1.9382e-06
Epoch 21/100
7s - loss: 2.3137e-06
Epoch 22/100
7s - loss: 2.2428e-06
Epoch 23/100
7s - loss: 1.7323e-06
Epoch 24/100
7s - loss: 2.2756e-06
Epoch 25/100
7s - loss: 2.2457e-06
Epoch 26/100
7s - loss: 2.6347e-06
Epoch 27/100
7s - loss: 1.8215e-06
Epoch 28/100
7s - loss: 1.6592e-06
Epoch 29/100
7s - loss: 1.488

In [104]:
# Load the data we make to use to make a prediction
new_option_df = pd.read_csv("keras_evaluation.csv")
new_option_scaled = scaler.transform(new_option_df)
scaled_evaluation_df = pd.DataFrame(new_option_scaled,columns=new_option_df.columns.values)

X = scaled_evaluation_df.drop('premium',axis=1).values

prediction = model.predict(X)

#prediction = prediction[0][0]
#prediction = scaler.inverse_transform(prediction)
prediction = prediction - scaler.min_[6]
prediction = prediction / scaler.scale_[6] 

print("Predicted premium is  ${}".format(prediction))

Predicted premium is  $[[  4.01811905e+01]
 [  2.42803618e-03]
 [  2.42803618e-03]
 ..., 
 [  2.88664341e-01]
 [  2.88664341e-01]
 [  2.88664341e-01]]
